## 5. Vagrant & Linux Servers

Vagrant concepts are useful for cloud computing.

So far, we've been able to:

* VM setup and Linux:

    - Vagrant cloud: vm images
    - Vagrantfile: vm settings
    - Vagrant commands: up, status, ssh, halt, destroy, etc.


### Vagrantfile IP, RAM & CPU

We will work on modifying the `Vagrantfile` inside the vm folders.

The `Vagrantfile` can be open using a text editor (e.g. sublime or notepad) and consists of a number of comments (which start each line with `#`) and the following lines without comments (Ruby script):

    ```
    Vagrant.configure("2") do |config|

        config.vm.box = "ubuntu/jammy64"

    end
    ```

There are some configurations in between which can be uncommented in order to apply new settings, for example:

``` 
# Lines 52-58
config.vm.provider "virtualbox" do |vb|

    vb.memory = "1048"  # (You can increase the memory size of the VM)

end
```

Another one is:

``` 
# Line 40
config.vm.network "public_network" # (Set bridge network interface: fetch IP address from your router randomly)
```

For CentOS, you need to add additional specification:

``` 
# Line 40
config.vm.network "public_network", bridge: "en0: Wi-Fi"
```

What if I need a static IP address?
```
# Line 35
config.vm.network "private_network", ip: "192.168.56.14"
```

We can also add additional settings:

``` 
# Lines 52-58
config.vm.provider "virtualbox" do |vb|

    vb.memory = "1048"  # (You can increase the memory size of the VM)
    vb.cpus = "2"
end
```

IMPORTANT: Once you've made all the changes in the Vagrantfile, save it and you need to reload the VM to apply them: `vagrant reload` inside the vagrant directory.

You can check the new settings when the vm is reloaded:

```
default: Adapter 1: nat
default: Adapter 2: hostonly
default: Adapter 3: bridged
```

And for the memory settings, login to the vm and run:

`free -m` in total mem.

and the cpu:

`cat /proc/cpuinfo` in cpu cores

### Vagrant Sync Directories

After running the command `vagrant up`, the following line appears in the log:

` default: /vagrant => G:/Mi unidad/GitHub/course-devops-udemy/vagrant-vms/ubuntu`

`/vagrant` is the path for the guest directory.

If you add a new file in `G:/Mi unidad/GitHub/course-devops-udemy/vagrant-vms/ubuntu`, it will be immediately synced in the guest directory `/vagrant` inside the vm. -> *sync directory*

Every file within these two directories is automatically syncronised.

We can also mention *our own path for the sync directory* by modifying the `Vagrantfile`:

```
# Line 46
config.vm.synced_folder "../data", "/vagrant_data"
```

You can set any path inside the guest machine, for example: `/opt/scripts` (gets created automatically) and you need to set the synced folder in Windows/Mac:

For Windows:

* Create folder o set the directories to be synced.
* Map the path of this directory using the following syntax (`\\`):
    
    `"F:\\scripts\\shellscripts"`, for example

For Mac:

* Create folder in the desktop
* Map the path using the following syntax (`/`):
    
    `"/Users/username/Desktop/scripts"`

You can have as many synced directories as you want in the Vagrantfile!

**Reasons to use synced directories**:

1. Preserve VM files into the host machine, in case it gets aborted or corrupted

2. Write your scripts/programs in the host machine by using the preferred editor

### Vagrant Provisioning

Provisioning is executing commands or scripts when the VM comes up first time or if the VM is already running by applying provisioning to execute commands or scripts.

Provising can be modified using the Vagrantfile:

```
# Line 66
config.vm.provision "shell", inline: <<-SHELL
    apt-get update
    apt-get install -y apache2
SHELL
```

In a CentOS VM, it should be:

```
# Line 66
config.vm.provision "shell", inline: <<-SHELL
    yum install httpd wget unzip git -y
SHELL
```

Provisioning is only executed only once. If you run `vagrant reload`, it will not execute the provisioning command.

However, you can force provision by running the command `vagrant provision` or the option `vagrant reload --provision`.